In [1]:
!pip install collie-bench nltk
import dill
import json
import logging
import os
import pandas as pd
import random
import sys
from typing import Any, Dict, List
from collie import constraints
from datetime import date
from dotenv import load_dotenv
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.utils import GPT


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# params
# st_mdl = "gpt-3.5-turbo-1106" # model string
st_mdl = "gpt-4-1106-preview"
st_task = "collie" # name of task

# logging
## today's date
st_today = date.today().strftime("%Y_%m_%d")

## file name:
st_fn = f"{st_today}_{st_task}_n200_{st_mdl}"

## Configure the logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Create a file handler and set the formatter
file_handler = logging.FileHandler(f"results/{st_fn}.log")
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

## Create a logger instance and add the file handler
logger = logging.getLogger(__name__)
logger.addHandler(file_handler)

In [9]:
# load api key from .env and create GPT object
load_dotenv()  # Load environment variables from .env file
openai_api_key = os.getenv("OPENAI_API_KEY")
gpt = GPT(openai_api_key, logger=logger, model=st_mdl)

# Load into a dictionary
with open("data/all_data.dill", "rb") as f:
    data: Dict[str, List[Dict[str, Any]]] = dill.load(f)
    # data.append(json.loads(f)) # keys in each row: ['_id', 'answer', 'question', 'supporting_facts', 'context', 'type', 'level']

# data = pd.read_csv('data/24game.csv').to_dict(orient='records') # load CSV into list of dicts

In [14]:
print(data['wiki_c14'][0].keys())

dict_keys(['example', 'metadata', 'targets', 'constraint', 'prompt'])


In [4]:
def compare_strings(str1, str2):
    """
    This function compares two strings after converting them to lowercase and stripping leading and trailing spaces.
    """
    return str1.lower().strip() == str(str2).lower().strip()


def check_mcq(pred, gold):
    """
        Check if prediction for multiple choice question is correct 
    """
    return pred.lower().strip()[0] == gold.lower()


def check_bool(str1, bool2):
    """
        Checks if string str1 equal to bool bool2
    """
    try:
        if bool(str1) == bool2:
            return True
    except:
        pass

    return False
        

In [5]:
# dataset-specific adjustments (we eventually want a list of dicts of format [{"id": "xx", "question": "xxx", "context": "xxx"}, ...]

DN_TASK_MAP = {
    'c01': 'word01',
    'c02': 'word02',
    'c03': 'word03',
    'c04': 'sent01',
    'c05': 'sent02',
    'c06a': 'sent03',
    'c07': 'sent04',
    'c08': 'para01',
    'c09': 'para02',
    'c10': 'para03',
    'c11': 'para04',
    'c12': 'para05',
    'c14': 'pass01'
}

def rename_dict_keys(original_dict, key_mapping):
    """
        Rename the keys of a dictionary based on a key_mapping dictionary. E.g.
        odict: {'text_c0': ['test'], 'text_c1': ['test']}
        kmap: {'c0': 'word_1'} 
        rename_dict_keys(odict, kmap): {'word_1': ['test'], 'test_c1': ['test']}

        original_dict: dictionary whose keys should be mapped
        key_mapping: mapping from old (including substrings of) keys to new ones
    """
    new_dict = {}
    
    for k, v in original_dict.items():
        # set new key = old key for now; we don't know if an actual new key exists
        nk = k
        for old_key, new_key in key_mapping.items():
            if old_key in k:
                nk = new_key
        if nk not in new_dict:
            new_dict[nk] = v.copy()
        else:
            new_dict[nk].extend(v.copy())
    
    return new_dict


data_names = list(data.keys())
data_names = sorted(data_names, key=lambda x: (x.split('_')[1], x.split('_')[0]))

# data containing only keys (dataset of origin name), and relevant constraint/
# prompt info (no text corpus)
d_no_text = {}
for dn in data_names:
    d_no_text[dn] = [{'c': ex['constraint'], 'p': ex['prompt'], \
                    't': ex['targets']} for ex in data[dn]]

data = rename_dict_keys(d_no_text, DN_TASK_MAP)
data_names = list(data.keys())

In [6]:
print(data['word02'][1]['t'])
print(list(data_names))

[13, ['d', 'p', 's']]
['word01', 'word02', 'word03', 'sent01', 'sent02', 'sent03', 'sent04', 'para01', 'para02', 'para03', 'para04', 'para05', 'pass01']


## Generator

In [7]:
predictions = [] # list of predictions, reset after each set of runs
indices = [] # list of question indices
answers = [] # list of gold answers
evals = [] # list of whether prediction matches answer
questions = [] # list of questions
lst_subtasks = [] # list of subtasks for each row
lst_indices = [] # list of indices for each row. These indices are subject-specific

# Evaluate
num_questions = 10 # number of questions to run per category
for dn in data_names:
    dn_data = data[dn]
    num_data = len(dn_data)
    indices = random.sample(range(num_data), min(num_data, num_questions))
    for idx in indices:
        item = dn_data[idx]
        lst_subtasks.append(dn)
        lst_indices.append(idx)
        constraint, question, target = item['c'], item['p'], item['t']
        
        logger.info(question)
        
        msg = [{"role": "system", "content": f"The following is a constraint satisfaction task. There may be multiple constraints but only a single answer should be produced. Please respond only with the answer, no additional text."}, {"role": "user", "content": question}]
        pred = gpt.gen(msgs=msg).content
        predictions.append(pred)
        evals.append(constraint.check(predictions[-1], target))
        questions.append(question)
        print(predictions[-1])

# Print the predictions and answers
logger.info(predictions)
# logger.info(answers)
percent_correct = (sum(evals) / len(evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# create a dataframe with the data 
df = pd.DataFrame({'Index': lst_indices, 'Subtask': lst_subtasks, 'Question': questions, 'Prediction': predictions, 'Eval': evals})

# save the datafrme to csv
df.to_csv(f'results/{st_fn}.csv', index=False)

2024-02-14 00:48:59,915 - INFO - Please generate a word with at least 12 characters.
2024-02-14 00:49:01,038 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 00:49:01,040 - INFO - Please generate a word with at least 13 characters.


Consequential


2024-02-14 00:49:01,608 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 00:49:01,610 - INFO - Please generate a word with at least 10 characters.


Misunderstanding


2024-02-14 00:49:02,100 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 00:49:02,102 - INFO - Please generate a word with at least 12 characters.


...

Our company's foundation stands on the unwavering commitment to excellence in everything we do, which is why our clients trust us with their most crucial needs. We are determined to be the best at doing what matters the most.

Years of experience have taught us that being just good is not good enough; we aim for perfection. We are intellectually honest in delivering quality, including on time delivery.

GOLDWOOD has always been at the forefront of innovation. From the selection of raw materials to the final product


2024-02-14 00:54:20,176 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 00:54:20,178 - INFO - Please generate a passage:
1) with paragraphs having the last sentence to be 'The NRA believes that devices designed to allow semi-automatic rifles to function like fully-automatic rifles should be subject to additional regulations,” they said', 'Legislators are now introducing a bill to deal with bump stocks, but many are saying that Congress must go far further', 'She’s executive director of the Nevada-based Battle Born Progress', 'Rarely has it ever said it would do anything, but, clearly, the pressure is so enormous, they are saying, “OK, let’s deal with this one thing.”', respectively;
2) with all paragraphs having at least 2 sentences.


Nestled amongst the serene mountains of Zhangjiakou, the area has become a burgeoning hub for transportation. The station hall started construction on to build an underground high-speed railway station on the Beijing–Zhangjiakou intercity railway (Part of Beijing–Baotou Passenger-Dedicated Line) opened on 30 December 2019.

Incorporating elements of local culture and aesthetics, the station's design aims to be more than a conduit for travelers. The architectural design


2024-02-14 00:54:26,248 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 00:54:26,250 - INFO - ['Consequential', 'Misunderstanding', 'Understandably', 'Knowledgeability', 'Confidential', 'Aberrations', 'Understanding', 'Understanding', 'Internationalization', 'Consideration', 'prevarication', 'miscomputation', 'individualisticly', 'lightingalestra', 'prevaricate', 'temptations', 'fantasticalbun', 'procrastinoid', 'mischievousness', 'knowledgeable', 'Superiorityx', 'considerationsx', 'frameworkz', 'complectrix', 'questioningz', 'considerationx', 'consideratex', 'Sure. This sentence has exactly ninety-two characters in it.', 'This sentence has been carefully crafted to contain exactly one hundred eleven characters.', 'This is an example sentence crafted to meet the ninety-four character limit.', 'Precisely crafting a sentence that fills up one hundred twelve characters is quite a task.', 'Today is a wonderful day to learn new things, app

In a sharp departure from its traditional stance, the National Rifle Association (NRA) has issued a statement on the latest mass shooting incident. "The NRA believes that devices designed to allow semi-automatic rifles to function like fully-automatic rifles should be subject to additional regulations,” they said.

The use of bump stocks—a device used by the shooter—has brought the gun control debate to the forefront of American politics. Legislators are now introducing a bill to deal with bump stocks, but many are saying that Congress
Percentage of True values: 47.24409448818898%


In [59]:
len(evals)

127

In [60]:
# df = pd.read_csv(f"results/{st_fn}.csv")
# df['Answer'] = df['Answer'].apply(lambda x: x.split('\n#### ')[-1])
# df['Eval'] = df.apply(lambda row: compare_strings(row['Prediction'], row['Answer']), axis=1)

# df.to_csv(f'results/{st_fn}.csv', index=False)
# df.head()

## Validator

In [7]:
gen_data_fn = f"results/{st_fn}.csv" # change this to the filename of the generator results
# gen_data_fn = 'results/2024_01_27_mmlu_n200_gpt-3.5-turbo-1106.csv'

df = pd.read_csv(gen_data_fn)
indices = df.iloc[:, 0].tolist()
indices = indices[:-1] # just because we manually added an additional row on the CSV when calculating scores

val_preds = [] # list of predictions for validator, reset after each set of runs
val_evals = [] # whether prediction matches answer


# Evaluate
for _, row in df.iterrows():
    ans = row['Prediction']
    gold = row['Eval']
    question = row['Question']
    
    q = "Question: " + question + " "
    q += f"\nAnswer: {ans}"
    logger.info(q)
    msg = [{"role": "system", "content": "The following is a constraint satisfaction problem. A single answer must be provided to satisfy one or more constraints. Respond 'TRUE' if the answer to the problem is correct and 'FALSE' if incorrect. Respond only with one word."}, {"role": "user", "content": q}]
    val_preds.append(gpt.gen(msgs=msg).content)
    val_evals.append(check_bool(val_preds[-1], gold))
    print(val_preds[-1])

# Print the predictions and answers
logger.info(val_preds)
logger.info(df['Eval'].tolist())
percent_correct = (sum(val_evals) / len(val_evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# add validator predictions to column
df['Validator Pred'] = val_preds

# save the dataframe to csv
df.to_csv(f'results/{st_fn}_with_val.csv', index=False);

2024-02-14 09:18:11,783 - INFO - Question: Please generate a word with at least 12 characters. 
Answer: Consequential
2024-02-14 09:18:12,340 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:18:12,342 - INFO - Question: Please generate a word with at least 13 characters. 
Answer: Misunderstanding


TRUE


2024-02-14 09:18:12,773 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:18:12,775 - INFO - Question: Please generate a word with at least 10 characters. 
Answer: Understandably


TRUE


2024-02-14 09:18:13,214 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:18:13,216 - INFO - Question: Please generate a word with at least 12 characters. 
Answer: Knowledgeability


...

FALSE


2024-02-14 09:19:41,791 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:19:41,793 - INFO - Question: Please generate a passage:
1) with paragraphs having the last sentence to be 'The NRA believes that devices designed to allow semi-automatic rifles to function like fully-automatic rifles should be subject to additional regulations,” they said', 'Legislators are now introducing a bill to deal with bump stocks, but many are saying that Congress must go far further', 'She’s executive director of the Nevada-based Battle Born Progress', 'Rarely has it ever said it would do anything, but, clearly, the pressure is so enormous, they are saying, “OK, let’s deal with this one thing.”', respectively;
2) with all paragraphs having at least 2 sentences. 
Answer: In a sharp departure from its traditional stance, the National Rifle Association (NRA) has issued a statement on the latest mass shooting incident. "The NRA believes that devices design

FALSE


2024-02-14 09:19:42,157 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 09:19:42,158 - INFO - ['TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'paths, setting a magical scene for evening strollers. Nightfall brought a peaceful silence, save for the occasional h

FALSE
Percentage of True values: 47.24409448818898%
